In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
import joblib
import pandas as pd
import numpy as np
from influxdb import DataFrameClient
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [3]:
scaler = joblib.load("../Models/rolling_nn3/scaler")

In [4]:
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[17], kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)),
    layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

In [5]:
model.load_weights("../Models/rolling_nn3/cp.ckpt")

In [6]:
client = DataFrameClient(host='localhost', port=8086)
client.get_list_database()

[{'name': '_internal'}, {'name': 'Megallen2'}]

In [7]:
client.switch_database('Megallen2')

In [8]:
res = client.query("SELECT * FROM OnlineRSSI WHERE time > now() - 3s")
if res == {}:
    raise Exception("No data incoming")
res = res["OnlineRSSI"]

In [9]:
B1 = res[res["beacon"] == "0117C55D14E4"]

In [10]:
B1

,beacon,rssi,scanner
2019-12-06 06:42:56.603000+00:00,0117C55D14E4,-70,DF231643E227
2019-12-06 06:42:56.603000+00:00,0117C55D14E4,-95,DB8B36A69C56
2019-12-06 06:42:56.603000+00:00,0117C55D14E4,-87,E43355CA8B96
2019-12-06 06:42:56.783000+00:00,0117C55D14E4,-73,E6D9D20DD197
2019-12-06 06:42:56.857000+00:00,0117C55D14E4,-79,E43355CA8B96
...,...,...,...
2019-12-06 06:42:59.273000+00:00,0117C55D14E4,-68,DF231643E227
2019-12-06 06:42:59.317000+00:00,0117C55D14E4,-87,D2B6503554D7
2019-12-06 06:42:59.339000+00:00,0117C55D14E4,-87,E43355CA8B96
2019-12-06 06:42:59.383000+00:00,0117C55D14E4,-71,E13B805C6CB0


In [11]:
B1 = B1.pivot(columns="scanner", values="rssi").rename_axis(None, axis=1)
B1

,C400A2E19293,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2019-12-06 06:42:56.603000+00:00,NaN,NaN,-95.0,NaN,-70.0,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.783000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.857000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.859000+00:00,NaN,NaN,NaN,NaN,-69.0,-81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.964000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06 06:42:59.273000+00:00,NaN,NaN,NaN,NaN,-68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.317000+00:00,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.339000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.383000+00:00,NaN,NaN,NaN,NaN,NaN,-71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
scanners = [
    'C400A2E19293', 'CD4533FFC0E1', 'D2B6503554D7', 'DB8B36A69C56', 'DD697EA75B68', 'DF231643E227', 
    'E13B805C6CB0', 'E43355CA8B96', 'E6D9D20DD197', 'E8FD0B453DC4', 'E96AF2C858BA', 'EC72840D9AD3', 
    'F1307ECB3B90', 'F1EDAF28E08A', 'F69A86823B96', 'FB2EE01C18CE', 'FDAE5980F28C'
]
B1 = B1.reindex(columns=scanners)
B1

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2019-12-06 06:42:56.603000+00:00,NaN,NaN,NaN,-95.0,NaN,-70.0,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.783000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.857000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.859000+00:00,NaN,NaN,NaN,NaN,NaN,-69.0,-81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.964000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06 06:42:59.273000+00:00,NaN,NaN,NaN,NaN,NaN,-68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.317000+00:00,NaN,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.339000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:59.383000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data = B1.rolling(50, min_periods=1).mean()
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2019-12-06 06:42:56.603000+00:00,NaN,NaN,NaN,-95.00,NaN,-70.000000,NaN,-87.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.783000+00:00,NaN,NaN,NaN,-95.00,NaN,-70.000000,NaN,-87.000000,-73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.857000+00:00,NaN,NaN,NaN,-95.00,NaN,-70.000000,NaN,-83.000000,-73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.859000+00:00,NaN,NaN,NaN,-95.00,NaN,-69.500000,-81.0,-83.000000,-73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 06:42:56.964000+00:00,NaN,NaN,NaN,-95.00,NaN,-69.500000,-81.0,-83.000000,-74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06 06:42:59.273000+00:00,-86.5,NaN,-84.333333,-86.20,-93.0,-66.285714,-73.5,-84.714286,-79.714286,-92.25,-70.25,-72.0,-71.0,NaN,-77.0,-84.0,-82.0
2019-12-06 06:42:59.317000+00:00,-86.5,NaN,-84.714286,-86.20,-93.0,-66.285714,-73.5,-84.000000,-79.714286,-92.25,-70.25,-72.0,-71.0,NaN,-77.0,-84.0,-82.0
2019-12-06 06:42:59.339000+00:00,-86.5,NaN,-84.714286,-86.20,-93.0,-66.285714,-73.5,-84.428571,-79.714286,-92.25,-70.25,-72.0,-71.0,NaN,NaN,-84.0,-82.0
2019-12-06 06:42:59.383000+00:00,-86.5,NaN,-84.714286,-85.75,-93.0,-66.285714,-73.0,-84.428571,-79.714286,-93.00,-70.25,-72.0,-71.0,NaN,NaN,-83.2,-82.0


In [14]:
data = data.ffill().fillna(-95)

In [15]:
data = scaler.transform(data)

In [16]:
pred = model.predict(data)

In [17]:
np.expand_dims(pred.mean(axis=0), axis=0)

array([[22.810469 , 10.0723095]], dtype=float32)